# Create Slice

Creates a simple slice with one node.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager(project_id="")
                     
fablib.show_config()

import json
import traceback

try:
    #Create Slice
    slice = fablib.new_slice(name="terminal")
    
    #Host
    host = slice.add_node(name="host", site="TACC")
    host.set_image("default_ubuntu_20")
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

# Terminal Class

In [ ]:
import ipywidgets as widgets
from ipywidgets import HTML, Layout

#This class will create an IPYWidgets terminal that will interface with a node
class terminal():
    #Class Variable
    node = None
    
    #Initializer, with a node input
    def __init__(self,node):
        self.node = node
        
        #Style
        display(widgets.HTML(value="<style>.inputt{border-color:black;}.out_box{ border: 1px solid black; height: 300px; width: 90%; overflow: auto;}</style>"))
        
        #Output
        self.out = widgets.Output()
        self.out.add_class('out_box')
        self.out.append_stdout("Connected to node: "+self.node.get_name()+"\n")
        display(self.out)
        
        #Input
        self.inputcmd = widgets.Text(layout=Layout(width='80%'))
        self.inputcmd.on_submit(self.submit)
        self.inputcmd.add_class('inputt')
        self.submitbtn = widgets.Button(description="submit",layout=Layout(width='10%'))
        
        self.inputs = widgets.HBox([self.inputcmd,self.submitbtn])
        
        display(self.inputs)
        self.submitbtn.on_click(self.submit)
        return
        
    #Send command to node
    def submit(self,*args):
        self.lastcmd = self.inputcmd.value
        self.inputcmd.value=""
        
        self.out.append_stdout("Input:"+self.lastcmd)
        self.out.append_stdout('\n')
        
        out,err = self.node.execute(self.lastcmd)
        
        self.out.append_stdout(out + " " + err)
        self.out.append_stdout('\n')
        
        return
    

# Call Terminal

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager(project_id="")

import json
import traceback

#Get node
slice = fablib.get_slice("terminal")
host = slice.get_node(name="host")

#Initialize terminal
term = terminal(host)